In [20]:
#imports
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import requests
import time 

#read in the html file

precinct_results = [] 
precinct_names = []
file_dict = {}

for i in range(1, 429):
    page_number = str(i)
    url = 'https://election.co.delaware.pa.us/eb/November_2022/reports/' + page_number + '.html'
    my_headers = {
    'Host' : 'e-com.secure.force.com',
    'Connection' : 'keep-alive',
    'Upgrade-Insecure-Requests' : '1',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64)',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding' : 'gzip, deflate, sdch',
    'Accept-Language' : 'en-US,en;q=0.8'
    }  
    page = requests.get(url, headers = my_headers).text
    soup = BeautifulSoup(page, 'html.parser')

    #selected by a class for this — not the tr — 
    #because the trs were giving me more info than I needed. 
    #You can search in the inspect bar for a634c to see what  
    #it gets— basically it's a whole table instance, which I'm  
    #effectively defining as the div that contains the table
    table_instances = soup.find_all(class_='a634c')
    precinct_name = soup.find_all(class_='a60l')[0].text
    precinct_names.append(precinct_name)
    file_dict[precinct_name] = i
    #for the first table instance, pull out the actual table. 
    #For a loop, you'd do: for table_instance in table_instances... 
    #and put table_instance in place of table_instances[0], etc.
    #Each table instance should have only one table that we're 
    #interested in, but as you test this out make sure

        #precinct_name = header[0].text
    
    dfs = [] 
    for table_instance in table_instances:
        table = table_instance.find_all('table')
        dfs.append(pd.read_html(str(table))[0])

    #save in data frame
    precinct_results.append(dfs)

In [3]:
#archive/ most elementary, unprocessed version of extract (timestamped)
from datetime import datetime
from datetime import date

now = datetime.now()
today = date.today()
current_time = now.strftime("%H%M")
current_date = today.strftime("%Y%m%d")

#precinct_results.to_csv("/Users/jmitovich/Downloads/municipality/delaware_{}_{}.csv".format(current_date, current_time))

428
5
11
11
8
8
9
9
9
9
9
9
9
9
9
9
0


KeyError: 0

In [21]:
municipalities = (pd.read_csv("/users/jmitovich/Downloads/delaware_municipalities.csv"))
precinct_dict = {}
for precinct in precinct_names:
    for i in range(0, 427):
        prec = municipalities.loc[i, "precinct"]
        muni = municipalities.loc[i, "muni_full"]
        precinct_dict[prec] = muni

muni_name = []
muni_fips = []
county_name = [] 
precinct_name = []
precinct_code = []
office = []
party = [] 
vote = [] 

for i in range(0, 427):
    name = municipalities.loc[i, "muni_full"]
    fips = municipalities.loc[i, "muni_fips"]
    cty = "delaware"
    indices = []
    val = municipalities.loc[i, "precinct"]
    if precinct_dict[val] == name:
        indices.append(file_dict[val])

    #all the precincts under that municipality 
    for j in indices: 
        precinct_df = precinct_results[j - 1]

        
        # go race by race for senate and gov
        for k in range(len(precinct_df))[0:2]:
        
            #go candidate by candidate 
            for l in range(len(precinct_df[k]))[3:-3]:
                
                muni_name.append(name)
                muni_fips.append(fips)
                county_name.append(cty)
                precinct_code.append(j)
                race_office = str(precinct_df[k][0][1])
                #office
                if "Senator" in race_office:
                    office.append("sen")
                elif "Governor" in race_office:
                    office.append("gov")
                else:
                    office.append("na")
                #party
                race_party = str(precinct_df[k][1][l])
                if "REP" in race_party:
                    party.append("rep")
                elif "DEM" in race_party:
                    party.append("dem")
                else:
                    party.append("oth")
                 
                #vote   
                race_vote = int(precinct_df[k][12][l])
                vote.append(race_vote)
                precinct_name.append(val)

                
precinct_totals = pd.DataFrame(data = {'muni_name': muni_name, 'muni_fips': muni_fips, 'county_name': county_name, 
                                       'precinct_name': precinct_name, 'precinct_code': precinct_code, 'office': office, 
                                       'party': party, 'vote': vote})
precinct_totals['vote'] = precinct_totals['vote'].fillna(0).astype('int')

# live/precinct/county (not timestamped)
precinct_totals.to_csv("/Users/jmitovich/Downloads/precinct/delaware.csv")

In [22]:
from datetime import datetime
from datetime import date
muni_grouped = precinct_totals.groupby(['muni_name', 'muni_fips', 'county_name', 'office', 'party']).agg({'vote': 'sum'})
muni_grouped.columns = ['vote']
muni_totals = muni_grouped.reset_index()

#live/municipality/county (not timestamped)
muni_totals.to_csv("/Users/jmitovich/Downloads/municipality/delaware.csv")